In [31]:
#Standard Python does not provide abstract classes, but module abc in standard library
from abc import ABC, abstractmethod
import pyodbc
import pandas as pd

class DBConnector(ABC):
    """Abstract class, inheriting from ABC
    But abstract classes can have implemented methods too"""
    def __init__(self:object , dbserver:str = None, dbname:str = None, dbusername: str = None, \
        dbpassword:str = None, DSN: str = None):
        #DSN means "Data Source Name", common acronym for a connection string
        self._dbserver = dbserver
        self._dbname = dbname
        self._dbusername = dbusername
        self._dbpassword = dbpassword #this should be obfuscated
        self._isDBConnectionOpen:bool = False
        self._conduit:pyodbc.Connection = None
        self._driver:str = 'undef'
            
        if(DSN is not None):
            self._DSN = DSN
        else:
            self._DSN = None

    @abstractmethod
    def selectBestDBDriverAvailable(self:object)->None:
        """
        This needs to be a C++ equivalent of a "pure virtual method"
        Will need implemented by the children of this classs, specialised in target data source, e.g. MS SQLSERVER (or any other)
        """
        #No syntax equivalent for virtual methodname() = 0; in C++, a Python method needs "code": only option is pass
        pass
    
    def Open(self: object):
        if(self._isDBConnectionOpen == False):
            if(self._conduit is None): #Use a property. Logic: if the pyodbc object is not yet instanciated
                try:
                    self._conduit = pyodbc.connect("DRIVER=" + self._driver \
                        +";SERVER="+self._dbserver+";DATABASE="+self._dbname+";UID="+self._dbusername+";PWD=" + self._dbpassword)

                    self._isDBConnectionOpen = True
                except Exception as excp:
                    raise Exception("Couldn't connect to the database").with_traceback(excp.__traceback__)
                    self._isDBConnectionOpen = False
            else: 
                raise Exception("Inconsistent state of the connector, flag set to not connected, conduit is not None")
        else:
            raise Exception("Inconsistent call to Open(), the connector is already connected to the data source")

    def Close(self:object)->None:
        if(self._isDBConnectionOpen == True):
            if(self._conduit): #Use a property. Logic: if the pyodbc object is not yet instanciated
                try:
                    self._isDBConnectionOpen = False
                    self._conduit = False #might be cursor.close(), need to be paid attention to.
                except Exception as excp:
                    raise Exception("Couldn't disconnect to the database").with_traceback(excp.__traceback__)
            else: 
                raise Exception("Inconsistent state of the connector, flag set to not connected, conduit is not exist")
        else:
            raise Exception("Inconsistent call to Close(), the connector is already disconnected to the data source")
    
    def ExecuteQuery_withRS(self:object, query:str):
        # Instantiate the object bearing the connection
        try:
            df = pd.read_sql(query, self._conduit)
            return df
        except Exception as excp:
            print("Something went wrong: " + str(excp)) 
    
    @property
    def dbServer(self:object)->str:
        return self._dbserver
    @dbServer.setter
    def dbServer(self:object, value:str)->None: #example of a type hint of return type to None --> means that it's a procedure
        self._dbserver = value

In [32]:
import sys
sys.path.append("C:/Users/daisu/OneDrive/Desktop/DSTI/SoftwareEngineering2/1_21")
from myTools import DBConnector as db
import pyodbc
import platform

class MSSQL_DBConnector(DBConnector): 
    def __init__(self:object , dbserver:str = None, dbname:str = None, dbusername: str = None, dbpassword:str = None, DSN: str = None):
        super().__init__(dbserver, dbname, dbusername, dbpassword, DSN)
        self.selectBestDBDriverAvailable()
        
    def selectBestDBDriverAvailable(self:object)->None:
        listOfAllAvailableDrivers:list[str] = pyodbc.drivers()
        identifiedOS:str = platform.system()
        if(listOfAllAvailableDrivers is not None):
            if("ODBC Driver 17 for SQL Server" in listOfAllAvailableDrivers):
                self._driver = "ODBC Driver 17 for SQL Server"
            if("ODBC Driver 13.1 for SQL Server" in listOfAllAvailableDrivers):
                self._driver = "ODBC Driver 13.1 for SQL Server"
    
        def fn_GetAllSurveyDataSQL():
        pass

In [41]:
mssql_connector = MSSQL_DBConnector(dbserver = "THISSIHT", dbname = "Survey_Sample_A19", dbusername ="sa", dbpassword = "249741")
mssql_connector.Open()

In [45]:
question:pd.DataFrame = mssql_connector.ExecuteQuery_withRS("SELECT * FROM dbo.Question")
answer:pd.DataFrame = mssql_connector.ExecuteQuery_withRS("SELECT * FROM dbo.Answer")
survey:pd.DataFrame = mssql_connector.ExecuteQuery_withRS("SELECT * FROM dbo.Survey")
structure:pd.DataFrame = mssql_connector.ExecuteQuery_withRS("SELECT * FROM dbo.SurveyStructure")
user:pd.DataFrame = mssql_connector.ExecuteQuery_withRS("SELECT * FROM dbo.User")

Something went wrong: Execution failed on sql 'SELECT * FROM dbo.User': ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Incorrect syntax near the keyword 'User'. (156) (SQLExecDirectW)")


In [1]:
# Objective:
#SQL
#1. to select data from tables
#2. to create/alter views

#Python
#1.Handle the connection to the database server
#2.Replicate the algorithm of the dbo.fn_GetAllSurveyDataSQL
#3.Replicate   the   algorithm   of   the   trigger dbo.trg_refreshSurveyViewfor creating/altering the view vw_AllSurveyDatawhenever applicable.
#4.For achieving (3) above, a persistence component (in any format you like: CSV, XML, JSON, etc.), storing the last known surveys’ structures should be in place. It is not acceptable  to  just  recreate  the  view  every  time: your Python  codereplacing  thetrigger behaviour must be as close as it can be, from “outside”the database.
#5.Of course, extract the “always-fresh”pivoted survey data, in a CSV file, adequately named.


# Import dependent libraries inside the script

#import pyodbc
def install(package):
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])


def connect_to_server():
    connectionString = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=THISSIHT;DATABASE=Survey_Sample_A19;UID=sa;PWD=249741"
    connectionString_Trusted = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=THISSIHT;DATABASE=Survey_Sample_A19;Trusted_Connection=yes"

    mssql_connection = pyodbc.connect(connectionString_Trusted)
    question = "SELECT * FROM dbo.Question"
    answer = "SELECT * FROM dbo.Answer"
    survey = "SELECT * FROM dbo.Survey"
    structure ="SELECT * FROM dbo.SurveyStructure"

    #SENDING A QUERY AND GETTING THE RESULTSET IN A PANDAS DATAFRAME
    df_question =pandas.read_sql(question, mssql_connection)
    df_answer = pandas.read_sql(answer, mssql_connection)
    df_survey = pandas.read_sql(survey, mssql_connection)
    df_structure = pandas.read_sql(structure, mssql_connection)
    print(df_question)
    print(df_answer)
    print(df_survey)
    print(df_structure)

    #### EXAMPLE - CONVERTING THE CODE OF THE STORED FUNCTION fn_GetAllSurveyDataSQL ####
    strQueryTemplateForAnswerColumn = ' \
        COALESCE( \
            ( \
                SELECT a.Answer_Value \
                FROM Answer as a \
                WHERE \
                    a.UserId = u.UserId \
                    AND a.SurveyId = <SURVEY_ID> \
                    AND a.QuestionId = <QUESTION_ID>\
            ), -1) AS ANS_Q<QUESTION_ID> '

        #### END OF EXAMPLE ####




# Example
if __name__ == '__main__':
    
    # Check if dependent libraries are installed. 
    #If installed, import all of them,
    #If not, install and import all of them
    
    import importlib
    for module in ["pandas", "pyodbc"]:
        try:
            importlib.import_module(module)
            print(f"{module} has been already installed \n")
            globals()[module] = importlib.import_module(module)
        except ModuleNotFoundError:
            print(f"{module} is not installed")
            print(f"Start installing {module}")
            install(module)
            print(f"End installing {module} \n")
            globals()[module] = importlib.import_module(module)
        else:
            continue
        
    
    # Connect to server
    try:
        connect_to_server()
    except Exception as excp:
        print("Something went wrong: " + str(excp))
        
        


pandas has been already installed 

pyodbc has been already installed 

   QuestionId                   Question_Text
0           1  What is your favourite colour?
1           2         Why did you chose DSTI?
2           3    Are you struggling with SQL?
3           4                  Test with Fred
      QuestionId  SurveyId   UserId  Answer_Value
0              1         1       42             3
1              1         1      296             5
2              1         1     2634             3
3              1         1     3714             7
4              1         1     6131             9
...          ...       ...      ...           ...
4994           3         2  1016263             6
4995           3         2  1017760             3
4996           3         2  1018956             8
4997           3         2  1020453             9
4998           3         2  1021179             7

[4999 rows x 4 columns]
   SurveyId        SurveyDescription  Survey_UserAdminId
0         1     

In [9]:
connectionString = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=THISSIHT;DATABASE=Survey_Sample_A19;UID=sa;PWD=249741"
connectionString_Trusted = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=THISSIHT;DATABASE=Survey_Sample_A19;Trusted_Connection=yes"

mssql_connection = pyodbc.connect(connectionString_Trusted)
question = "SELECT * FROM dbo.Question"
answer = "SELECT * FROM dbo.Answer"
survey = "SELECT * FROM dbo.Survey"
structure ="SELECT * FROM dbo.SurveyStructure"
user = "SELECT * FROM dbo.User"

#SENDING A QUERY AND GETTING THE RESULTSET IN A PANDAS DATAFRAME
df_question =pandas.read_sql(question, mssql_connection)
df_answer = pandas.read_sql(answer, mssql_connection)
df_survey = pandas.read_sql(survey, mssql_connection)
df_structure = pandas.read_sql(structure, mssql_connection)
df_user = pandas.read_sql(user, mssql_connection)
print(df_question)
print(df_answer)
print(df_survey)
print(df_structure)

DatabaseError: Execution failed on sql 'SELECT * FROM dbo.User': ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Incorrect syntax near the keyword 'User'. (156) (SQLExecDirectW)")

In [3]:
df_question

,QuestionId,Question_Text
0,1,What is your favourite colour?
1,2,Why did you chose DSTI?
2,3,Are you struggling with SQL?
3,4,Test with Fred


In [4]:
df_survey

,SurveyId,SurveyDescription,Survey_UserAdminId
0,1,SurveyA,2.0
1,2,SurveyB,NaN
2,3,Why is Seb so annoying?,1.0


In [5]:
df_answer

,QuestionId,SurveyId,UserId,Answer_Value
0,1,1,42,3
1,1,1,296,5
2,1,1,2634,3
3,1,1,3714,7
4,1,1,6131,9
...,...,...,...,...
4994,3,2,1016263,6
4995,3,2,1017760,3
4996,3,2,1018956,8
4997,3,2,1020453,9


In [7]:
df_structure

,SurveyId,QuestionId,OrdinalValue
0,1,1,1
1,1,2,2
2,2,2,1
3,2,3,2
4,3,1,1


In [26]:
import pandas as pd

question_structure = df_structure.merge(df_question, on="QuestionId", how="left")
question_structure

,SurveyId,QuestionId,OrdinalValue,Question_Text
0,1,1,1,What is your favourite colour?
1,1,2,2,Why did you chose DSTI?
2,2,2,1,Why did you chose DSTI?
3,2,3,2,Are you struggling with SQL?
4,3,1,1,What is your favourite colour?


In [29]:
question_survey_structure = question_structure.merge(df_survey, on="SurveyId", how="left")

In [36]:
question_survey_structure

,SurveyId,QuestionId,OrdinalValue,Question_Text,SurveyDescription,Survey_UserAdminId
0,1,1,1,What is your favourite colour?,SurveyA,2.0
1,1,2,2,Why did you chose DSTI?,SurveyA,2.0
2,2,2,1,Why did you chose DSTI?,SurveyB,NaN
3,2,3,2,Are you struggling with SQL?,SurveyB,NaN
4,3,1,1,What is your favourite colour?,Why is Seb so annoying?,1.0


In [49]:
df_full = question_survey_structure.merge(df_answer, on=["SurveyId", "QuestionId"], how="left")

In [50]:
df_full.SurveyDescription.value_counts()

SurveyA                    1676
SurveyB                    1675
Why is Seb so annoying?    1648
Name: SurveyDescription, dtype: int64

In [51]:
df_full

,SurveyId,QuestionId,OrdinalValue,Question_Text,SurveyDescription,Survey_UserAdminId,UserId,Answer_Value
0,1,1,1,What is your favourite colour?,SurveyA,2.0,42,3
1,1,1,1,What is your favourite colour?,SurveyA,2.0,296,5
2,1,1,1,What is your favourite colour?,SurveyA,2.0,2634,3
3,1,1,1,What is your favourite colour?,SurveyA,2.0,3714,7
4,1,1,1,What is your favourite colour?,SurveyA,2.0,6131,9
...,...,...,...,...,...,...,...,...
4994,3,1,1,What is your favourite colour?,Why is Seb so annoying?,1.0,1019491,1
4995,3,1,1,What is your favourite colour?,Why is Seb so annoying?,1.0,1020147,3
4996,3,1,1,What is your favourite colour?,Why is Seb so annoying?,1.0,1021182,9
4997,3,1,1,What is your favourite colour?,Why is Seb so annoying?,1.0,1021328,5


In [57]:
df_full[df_full["UserId"] ==1793]

,SurveyId,QuestionId,OrdinalValue,Question_Text,SurveyDescription,Survey_UserAdminId,UserId,Answer_Value
888,1,2,2,Why did you chose DSTI?,SurveyA,2.0,1793,6


In [66]:
filter1 = df_full.SurveyId == 1
filter2 = df_full.QuestionId
df_full[filter1]

,SurveyId,QuestionId,OrdinalValue,Question_Text,SurveyDescription,Survey_UserAdminId,UserId,Answer_Value
0,1,1,1,What is your favourite colour?,SurveyA,2.0,42,3
1,1,1,1,What is your favourite colour?,SurveyA,2.0,296,5
2,1,1,1,What is your favourite colour?,SurveyA,2.0,2634,3
3,1,1,1,What is your favourite colour?,SurveyA,2.0,3714,7
4,1,1,1,What is your favourite colour?,SurveyA,2.0,6131,9
...,...,...,...,...,...,...,...,...
1671,1,2,2,Why did you chose DSTI?,SurveyA,2.0,1011541,6
1672,1,2,2,Why did you chose DSTI?,SurveyA,2.0,1011746,4
1673,1,2,2,Why did you chose DSTI?,SurveyA,2.0,1014241,2
1674,1,2,2,Why did you chose DSTI?,SurveyA,2.0,1015832,0
